# Trabalho #4 - Data pipeline 


Nesse trabalho você vai utilizar um conjunto de dados estruturados para realizar uma tarefa de classificação multiclasse. Para isso, você vai utilizar o  módulo `feature_columns` para criar um pipeline de entrada para a RNA. 

As etapas que você vai realizar para finalizar esse trabalho são as seguintes:

1. Carregar um arquivo CSV usando Pandas;
2. Construir um pipeline de entrada;
3. Transformar as colunas de dados usando o módulo `feature_columns`;
4. Construir, treinar e avaliar uma RNA.

O conjunto de dados utilizado nesse trabalho consiste de uma versão simplificada do conjunto "PetFinder.my Adoption Prediction", que está disponível no Kaggle em https://www.kaggle.com/c/petfinder-adoption-prediction.

## Coloque seu nome aqui:

Nome: 

## Importar principais bibliotecas e funções

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## 1. Objetivo do trabalho

As instruções dadas ao longo do enunciado desse trabalho consistem apenas de indicações para facilitar o desenvolvimento do seu trabalho e não precisam ser seguidas exatamente como são fornecidas. 

Você vai verificar que se seguir exatamente as instruções para realizar esse trabalho os resultados obtidos serão insatisfatórios.

O que é esperado de você nesse trabalho é implementar um pipeline de dados e uma RNA que apresenta um desempenho nos resultados de teste melhor do que o obtido simplesmente seguindo as instruções dadas. Assim, você deve tentar realizar modificações no que é sugerido para atingir resultados satisfatórios.

## 2. Conjunto de dados

### 2.1 Descrição dos dados

Existem mais de 11.000 de exemplos (linhas) nesse conjunto de dados, que está na forma de um arquivo tipo CSV. Cada linha descreve um animal de estimação e cada coluna descreve uma característica do animal. Usaremos essas informações para prever a velocidade de adoção de um animal em função de suas características.

A seguir está uma descrição deste conjunto de dados. Observe que existem colunas numéricas e categóricas. Existe também uma coluna de texto livre que não será usada nesse trabalho.

**Entradas:**

- Type - Type of animal (1 = Dog, 2 = Cat)
- Age - Age of pet when listed, in months
- Breed1 - Primary breed of pet (307 valores - Refer to BreedLabels dictionary)
- Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
- Color1 - Color 1 of pet (7 values - Refer to ColorLabels dictionary)
- Color2 - Color 2 of pet (7 values - Refer to ColorLabels dictionary)
- MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
- FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
- Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
- Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
- Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
- Fee - Adoption fee (0 = Free)
- Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese
- PhotoAmt - Total uploaded photos for this pet

**Saída:**

- AdoptionSpeed
   - 0 - Pet was adopted on the same day as it was listed.
   - 1 - Pet was adopted between 1 and 7 days (1st week) after being listed.
   - 2 - Pet was adopted between 8 and 30 days (1st month) after being listed.
   - 3 - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.
   - 4 - No adoption after 100 days of being listed. (There are no pets in this dataset that waited between 90 and 100 days).

### 2.2 Carregar dados

Esse conjunto de dados está disponível no TensorFlow Dataset e são carregados de uma URL. Usaremos o Pandas para carregar o arquivo CSV em um DataFrame. Execute a célula abaixo para carregar os dados.

In [ ]:
# URL dos dados
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'

# Nome do arquivo de dados
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

# Carrega arquivo de dados
dataframe = pd.read_csv(csv_file)

# Dimensão dos dados
print('Dimensão dos dados:', dataframe.shape)
dataframe

- Observe que os dados consitem de 11.537 exemplos (linhas) e 15 características (colunas).

## 3. Pré-processamento dos dados

Antes de criar o pipeline de entrada dados, devemos verificar se as classes são balanceadas, remover a coluna de texto ("Description") que não vamos utilizar e dividir os dados nos conjuntos de treinamento, validação e teste.


### 3.1 Analisar saídas

Para verificar se as saídas são balanceadas temos que contar o número de exemplos de cada classe. Execute a célula abaixo para realizar essa contagem.

In [ ]:
print('Classes existentes nas saídas:', dataframe.AdoptionSpeed.unique())
print('Número de exemplos de cada classe:\n', dataframe.AdoptionSpeed.value_counts())

### Exercício #1: 

O número de exemplos da classe 0 é muito pequeno em relação às outras classes, assim, vamos unir os exemplos da classe 0 com os da classe 1. Fazendo isso, teremos as seguintes saídas possíveis:

    0 - Animal foi adotado entre o dia em que foi listado e 7 dias (1ª semana) após ser listado.
    1 - Animal foi adotado entre 8 e 30 dias (1º mês) após ser listado.
    2 - O animal foi adotado entre 31 e 90 dias (2º e 3º mês) após ser listado.
    3 - Sem adoção após 100 dias de listagem.

Na célula abaixo modifique a coluna "AdoptionSpeed" do DataFrame de forma a alterar os índices das classes de acordo com o definido acima. Para isso use o método `replace` do Pandas com o argumento `inplace-=True`.

In [ ]:
# Para você fazer: Alterar classes

# Modifique a coluna AdoptionSpeed


# Mostra novas classes e contagem de exemplos de cada classe
print('Classes existentes nas saídas:', dataframe.AdoptionSpeed.unique())
print('Número de exemplos de cada classe:\n', dataframe.AdoptionSpeed.value_counts())

**Saída esperada:**

    Classes existentes nas saídas: [1 0 2 3]
    Número de exemplos de cada classe:
    1    3153
    3    3080
    0    2761
    2    2543
    Name: AdoptionSpeed, dtype: int64

- Observe que agora o número de exemplos das classes está mais balanceado.

### Exercício #2: Remover coluna de texto e dividir dados 

Na célula abaixo crie um código que realiza duas operações:

1. Remoção da coluna "Description"
2. Divisão do conjunto de dados em treinamento, validação e teste.

Nessa divisão usaremos 70% dos dados para treinamento, 15% para validação e 15% para teste.

In [ ]:
# Para você fazer: 

# Remoção da coluna "Description


# Divisão dos dados



print('Numéro de exemplos de treinamento:', len(train))
print('Numéro de exemplos de validação:', len(val))
print('Numéro de exemplos de teste:', len(test))

**Saída esperada:**

    Numéro de exemplos de treinamento: 8075
    Numéro de exemplos de validação: 1731
    Numéro de exemplos de teste: 1731

Execute a célula abaixo para calcular as estatísticas dos dados numéricos.

In [ ]:
dataframe.describe().transpose()

## 4. Criar pipeline de entrada de dados

Para criar um pipeline de entradas de dados usando as ferramentas do tensorFlow deve-se realizar as seguintes etapas:

1. Criar objeto dataset
2. Aplicar transformações desejadas nas colunas de dados
3. Escolher quais colunas serão usadas como entrada da RNA


### 4.1 Criar objetos dataset

Para utilizar o módulo `feature_columns` para transformar os dados de entrada enquanto são carregados na RNA durante o treinamento, eles devem estar em um objeto dataset do TensorFlow. 


### Exercício #3: Criar dataset para pipeline de entrada

Na célula abaixo crie uma função que realiza as seguintes operações:

1. Copia dataframe para um novo dataframe sem vínculo com o anterior usando o método copy()
2. Separa a coluna de saídas do dataframe
3. Cria dataset com método `tf.data.Dataset.from_tensor_slices()` usando as entradas e as saídas do DataFrame copiad
4. Embaralha dados se for desejado

Após criar essa função crie os datasets com os dados de treinamento, validação e teste. Use `shuffle=True` para os dados de treinamento e `shuffle=False` para os dados de validação e teste.

In [ ]:
# Para você fazer: Função para criar objeto dataset do TensorFlow

# Define função usando o método tf.data.Dataset.from_tensor_slices
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    # Copia dataframe sem vínculo com o anterior usando método copy()

    
    # Separa saídas do dataframe

    
    # Cria dataset com método tf.data.Dataset.from_tensor_slices usando o DataFrame com as entradas e as saídas

    
    # verifica se deve embaralhar dados
    if shuffle:

    
    # Cria lote de dados usando variável batch_size
    
    
    # Retorna dataset
    return ds

# Define o tamanho do lote de exemplos
batch_size = 32

# Cria dataset de treinamento, validação e teste, para isso use:
# shuffle=True para os dados de treinamento e shuffle=Flase para os dados de validação e teste


Execute a célula abaixo para criar um lote de dados e verificar se o seu código está correto. Note que somente os 5 primeiros exemplos são mostrados.

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('Lista das características:', list(feature_batch.keys()))
    print('Um lote de tipo:', format(feature_batch['Type'][:5]))
    print('Um lote de idade:', format(feature_batch['Age'][:5]))
    print('Um lote de Breed1:', format(feature_batch['Breed1'][:5]))
    print('Um lote de Gender:', format(feature_batch['Gender'][:5]))
    print('Um lote de Color1:', format(feature_batch['Color1'][:5]))
    print('Um lote de Color2:', format(feature_batch['Color2'][:5]))
    print('Um lote de MaturitySize:', format(feature_batch['MaturitySize'][:5]))
    print('Um lote de FurLength:', format(feature_batch['FurLength'][:5]))  
    print('Um lote de Vaccinated:', format(feature_batch['Vaccinated'][:5]))   
    print('Um lote de Sterilized:', format(feature_batch['Sterilized'][:5]))
    print('Um lote de Health:', format(feature_batch['Health'][:5]))
    print('Um lote de Fee:', format(feature_batch['Fee'][:5]))  
    print('Um lote de PhotoAmt:', format(feature_batch['PhotoAmt'][:5]))   
    print('Um lote de saída:', format(label_batch[:5]))

**Saída desejada:**

    Lista das características: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
    Um lote de idade: [ 1  3 36  4 36]
    Um lote de Breed1: [b'Mixed Breed' b'Rottweiler' b'Mixed Breed' b'Mixed Breed' b'Mixed Breed']
    Um lote de Gender: [b'Female' b'Male' b'Male' b'Female' b'Male']
    Um lote de Color1: [b'Black' b'Black' b'Brown' b'Brown' b'Brown']
    Um lote de Color2: [b'No Color' b'Yellow' b'No Color' b'No Color' b'No Color']
    Um lote de MaturitySize: [b'Medium' b'Medium' b'Medium' b'Medium' b'Medium']
    Um lote de FurLength: [b'Long' b'Medium' b'Medium' b'Medium' b'Short']
    Um lote de Vaccinated: [b'Yes' b'Yes' b'Not Sure' b'No' b'Yes']
    Um lote de Sterilized: [b'No' b'No' b'Yes' b'No' b'Yes']
    Um lote de Health: [b'Healthy' b'Healthy' b'Minor Injury' b'Healthy' b'Minor Injury']
    Um lote de Fee: [0 0 0 0 0]
    Um lote de PhotoAmt: [3 4 1 3 2]
    Um lote de saída: [3 3 4 4 4]

- Pode-se ver que o dataset retorna um dicionário de nomes das colunas de dados e um lote de cada característica.

- Observa-se que nesse conjunto de dados a maioria das caracteríticas são do tipo categóricas.

### 4.2 Função auxílar para verificar os resultados

Para verificar se as transfromações que estamos implementando estão corretas temos que criar uma função para isso, como visto em aula.

Execute a célula abaixo para criar um lote para ser usado como exemplo e criar a função para visualizar o resultado de uma transformação. Note que somente os 5 primeiros exemplos do lote serão mostrados.

In [ ]:
# Lote para ser usado como exemplo
example_batch = next(iter(train_ds))[0]

# Função auxiliar para visualizar resulatdos de transfromações
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy()[:5])

### 4.3 Transformação e inclusão das colunas

Para os dados poderem ser usados por uma RNA eles devem ser transformados. 

Após definir cada uma das transformações temos que incluir as colunas que serão usadas pela RNA em uma lista.

Execute a célula abaixo para inciar a lista de colunas que serão usadas pela RNA e o numéro de entradas total de cada exemplo, que é a soma das entradas de cada dado transformado.

In [ ]:
# Incializa lista de características desejadas
columns_list = []

# Inicializa número de entradas de cada exemplo
num_input = 0

### 4.4 Transformação das colunas "Age" e "PhotoAmt"
 
A coluna "Age" representa a idade dos animais em meses. Essa coluna é de dados numéricos, que variam de 0 a 255 meses. 

A coluna "PhotoAmt" representa o número de fotos de cada animal. Essa coluna é de dados numéricos, que variam de 0 a 30.

Tanto a idade como o número de fotos exatos de um animal não são muito importantes. Dessa forma, para utilizar essas duas colunas vamos segmentá-las em intervalos.

Para a idade vamos utilizar em 6 intervalos:

    idade < 3
    3 <= idade < 6
    6 <= idade < 12
    12 <= idade < 60
    60 <= idade < 120
    idade >= 120
    
Para o número de fotos vamos utilizar 4 intervalos:

    fotos < 1
    1 <= fotos < 3
    3 <= fotos < 5
    fotos >= 5


### Exercício #4: Segmentação das colunas "Age" e "PhotoAmt"

Para segmentar uma coluna numérica usando o módulo `feature_column`, é necessário primeiramente definir a coluna como sendo numérica usando o método `feature_column.numeric_column`, para depois segmentá-la usando o método `feature_column.bucketized_column`. 

Na célula abaixo crie um código que segmenta a coluna "Age" nos 6 intervalos definidos acima.

In [ ]:
# Para você fazer: Segmentação da coluna Age

# Define coluna como sendo de dados numéricos


# Segmenta coluna


# Mostra resultado
demo(animal_age_segs)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:** 

    [[1. 0. 0. 0. 0. 0.]
     [0. 1. 0. 0. 0. 0.]
     [0. 0. 0. 1. 0. 0.]
     [0. 1. 0. 0. 0. 0.]
     [0. 0. 0. 1. 0. 0.]]

Na célula abaixo crie um código que segmenta a coluna "PhotoAmt" nos 4 intervalos definidos acima.

In [ ]:
# Para você fazer: Segmentação da coluna PhotoAmt

# Define coluna como sendo de dados numéricos


# Segmenat coluna


# Mostra resultado
demo(animal_photo_segs)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input +=

**Saída esperada:** 

    [[0. 0. 1. 0.]
     [0. 0. 0. 1.]
     [0. 0. 0. 1.]
     [0. 1. 0. 0.]
     [0. 0. 1. 0.]]

### 4.5 Transformação da coluna "Fee"

A coluna "Fee" representa a taxa paga para adotar um determinado animal. Essa coluna é de dados numéricos, sendo que seus valores de variam de 0 a 2000. Para poder ser usado pela RNA esses dados devem ser normalizados. A normalização que iremos utilizar para essa  coluna é transformar os dados para valores reais no intervalo de 0 a 1.

### Exercício #5: Normalização da coluna "Fee"

Para normalizar a coluna "Fee", ao transformá-la em uma coluna numérica selecionamos uma função para aplicar a normalização desejada.

Na célula abaixo crie um código para fazer essas operações.

In [ ]:
# Para você fazer: Normalização da coluna Fee

# Calcula valor máximo da coluna

print('Fee máxima =', max_fee, '\n')

# Cria função para normalização
def norm_max_fee(x):

    
# Aplica método tf.feature_column.numeric_column usando função norm_max_fee


# Mostra resultado
demo(animal_fee_norm)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    Fee máxima = 2000 

    [[0. ]
     [0. ]
     [0. ]
     [0. ]
     [0.1]]

### 4.6 Transformação das colunas de dados categóricos

As colunas de dados categóricos desse conjunto de dados são as seguintes:

1. Type;
2. Color1;
3. Color2;
4. Gender;
5. MaturitySize;
6. Furlength;
7. vaccinated;
8. Sterilized;
9. Health.

### Exercício #6: Transformação das colunas de dados categóricos

Transforme todas essas colunas em dados categóricos e depois em vetores one-hot.

- Para transformar as colunas em dados categóricos use o método `feature_column.categorical_column_with_vocabulary_list`. Para passar a lista de termos pode usar por exemplo, para a coluna "Type" `dataframe.Type.unique()`;


- Para codificar as categorias em vetores one-hot use o método `feature_column.indicator_column`.


In [ ]:
# Para você fazer: Coluna Type

# Coluna Type


# Mostra resultado
demo(animal_type_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**saída esperada:**

    [[1. 0.]
     [1. 0.]
     [1. 0.]
     [0. 1.]
     [0. 1.]]

In [ ]:
# Para você fazer: Coluna Color1
    
# Coluna Color1


# Mostra resultado
demo(animal_color1_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 1. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 1.]
     [0. 0. 0. 0. 0. 1. 0.]
     [0. 0. 0. 0. 1. 0. 0.]]

In [ ]:
# Para você fazer: Coluna Color2
    
# Coluna Color2


# Mostra resultado
demo(animal_color2_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0. 0. 0. 0. 0.]
     [1. 0. 0. 0. 0. 0. 0.]
     [1. 0. 0. 0. 0. 0. 0.]
     [0. 0. 1. 0. 0. 0. 0.]
     [0. 0. 1. 0. 0. 0. 0.]]

In [ ]:
# Para você fazer: Coluna Gender
    
# Coluna Gender


# Mostra resultado
demo(animal_gender_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[0. 1.]
     [0. 1.]
     [1. 0.]
     [1. 0.]
     [1. 0.]]

In [ ]:
# Para você fazer: Coluna MaturitySize
    
# Coluna MaturitySize


# Mostra resultado
demo(animal_MaturitySize_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[0. 1. 0.]
     [1. 0. 0.]
     [0. 0. 1.]
     [0. 1. 0.]
     [0. 0. 1.]]

In [ ]:
# Para você fazer: Coluna FurLength
    
# Coluna FurLength


# Mostra resultado
demo(animal_FurLength_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0.]
     [0. 0. 1.]
     [0. 1. 0.]
     [0. 1. 0.]
     [1. 0. 0.]]

In [ ]:
# Para você fazer: Coluna Vaccinated
    
# Coluna Vaccinated


# Mostra resultado
demo(animal_Vaccinated_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0.]
     [0. 0. 1.]
     [0. 0. 1.]
     [1. 0. 0.]
     [0. 0. 1.]]

In [ ]:
# Para você fazer: Coluna Sterilized
    
# Coluna Sterilized


# Mostra resultados
demo(animal_Sterilized_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0.]
     [0. 0. 1.]
     [0. 1. 0.]
     [1. 0. 0.]
     [1. 0. 0.]]

In [ ]:
# Para você fazer: Coluna Health
    
# Coluna Health


# Mostra resultados
demo(animal_Health_one_hot)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[1. 0. 0.]
     [1. 0. 0.]
     [1. 0. 0.]
     [1. 0. 0.]
     [1. 0. 0.]]

### 4.7 Transformação da coluna "Breed1"

A coluna que fornece a raça ("Breed1") do animal possui 307 termos diferentes. Transformar essa coluna em vetores one-hot criaria colunas com muitos zeros, o que seria inconveniente. Portanto, nesse caso é melhor utilizar a codificação "embedding" para descrever esses 307 termos diferentes que representam as possíveis raças dos animais. 

Ressalta-se que usar uma codificação "embedding" é melhor para representar dados categóricos quando se tem muitas categorias possíveis. 


### Exercício #7: Transformação da coluna "Breed1"

Na célula abaixo crie um código que realiza a codificação "embedding" da  coluna "Breed'". Observa-se que o tamanho dos vetores "embedding" é um parâmetro que deve ser ajustado. Nesse trabalho tentaremos usar vetores com 8 elementos.

Para fazer essa transformação você deve primeiramente transformar os dados em categorias, usando o método `feature_column.categorical_column_with_vocabulary_list`, para depois realizar a codificação "embeding" usando o método `feature_column.embedding_column`. 

In [ ]:
# Para você fazer:

# Transformação da coluna Breed1 em categórica


# Transfromação em vetores "embedding"


# Mostra resultados
demo(breed1_embedding)

# Inclui coluna na lista de características desejadas


# Atualiza número de entradas
num_input += 

**Saída esperada:**

    [[-0.42777395  0.27999237  0.24018171  0.07493864  0.2034157  -0.09776279
       0.18314685  0.04246866]
     [ 0.22655667  0.16137254  0.33353806  0.21287134  0.27651897 -0.16279218
      -0.25303188 -0.33496138]
     [-0.07981153  0.10738403 -0.27436027 -0.1045252   0.11560448 -0.37660906
      -0.13173237 -0.29564175]
     [-0.3764975  -0.2668523  -0.11631445 -0.15872143  0.23671539 -0.12712602
      -0.2112993   0.30894685]
     [-0.25282273  0.52100515  0.03996713 -0.5479238   0.5706844  -0.25294143
       0.5259882  -0.42650932]]
       
- Observe que os valores dos elementos dos vetores "embedding" são aprendidos durante o treinamento do RNA. Esses valores nesse momento são números aleatórios utilizados para incializar os vetores.

### 4.8 Verificação da lista de colunas

Execute a célula abaixo para verificar se a lista de características usadas pela RNA está correta e o número de entradas de cada exemplo.

In [ ]:
print('Número de entradas =', num_input)
print('\nLista de colunas de características:')
for i in range(13):
    print(i+1, '-', columns_list[i])

**Saída esperada:**

    Número de entradas = 52

    Lista de colunas de características:
    1 - BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(3, 6, 12, 60, 120)) 
    2 - BucketizedColumn(source_column=NumericColumn(key='PhotoAmt', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(1, 3, 5)) 
    .
    .
    .
    12 - IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Health', vocabulary_list=('Healthy', 'Minor Injury', 'Serious Injury'), dtype=tf.string, default_value=-1, num_oov_buckets=0)) 
    13 - EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='Breed1', vocabulary_list=('Tabby', 'Domestic Medium Hair', 'Mixed Breed', 'Domestic Short Hair', 'Domestic Long Hair', 'Terrier', 'Persian', 'Rottweiler', 'Jack Russell Terrie ... 

## 5. Criar e compilar a RNA

Após definir as tranformações desejadas nos dados, vamos criar a RNA para realizar a tarefa de classificação multiclasse para prever a velocidade de adoção dos animais.


### Exercício #8: Configurar RNA

Para configurar a RNA, a primeira etapa é definir uma camada tipo `DenseFeatures` para inserir as colunas de características selecionadas na variável `columns_list`.

A RNA que iremos usar incicialmente tem a seguinte arquitetura:

1. Uma camada tipo `DenseFeatures`;
2. Uma camada intermediária tipo `Dense`, com 128 neurônios e função de ativação ReLu;
3. Uma camada intermediária tipo `Dense`, com 128 neurônios e função de ativação ReLu;
4. Uma camada de saída tipo `Dense`, com 5 neurônios e função de ativação softmax.

Observe que temos 4 classes de velocidade de adoção, cujos índices variam de 0 a 3.

Realize essas operações na célula abaixo.

In [ ]:
# Para você fazer: Configurar a RNA

# Cria camada DenseFeatures


# Configura RNA


# Sumário da rede
rna.summary

**Saída esperada:**

    <bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x00000200A691F9C8>>

### Exercício #9: Compilar RNA

Para treinar a RNA tente usar inicialmente os seguintes parâmetros:

- Otimizador: SGD
- Taxa de aprendizado: 0.01
- Função de custo: SparseCategoricalCrossentropy
- Métrica: Accuracy

Na célula abaixo compile a sua RNA com esses parâmetros.

In [ ]:
# Para você fazer: Compilar RNA

# Compila RNA


## 6. Treinar e avaliar a RNA

### Exercício #10: Treinar a RNA

Na célula abaixo treine a sua RNA usando 50 épocas. Use os dados de validação para verificar se está ocorrendo problemas de overfitting.

In [ ]:
# Para você fazer: 

# Treinar a RNA



**Saída esperada:**

    Epoch 50/50
    253/253 [==============================] - 1s 4ms/step - loss: 0.5468 - accuracy: 0.7777 - val_loss: 2.4025 - val_accuracy: 0.3403

Execute a célula abaixo para verificar como foi o processo de treinamento da sua RNA.

In [ ]:
# Recupera resultados de treinamento do dicinário history
acc      = results.history['accuracy']
val_acc  = results.history['val_accuracy']
loss     = results.history['loss']
val_loss = results.history['val_loss']

# Cria vetor de épocas
epocas   = range(len(acc)) 

# Gráfico dos valores da função de custo
plt.plot(epocas, loss, 'r', label='Custo - treinamento')
plt.plot(epocas, val_loss, 'b', label='Custo - validação')
plt.title('Valor da função de custo – treinamento e validação')
plt.xlabel('Épocas')
plt.ylabel('Custo')
plt.legend()
plt.show()

# Gráfico dos valores da métrica
plt.plot(epocas, acc, 'r', label='exatidao- treinamento')
plt.plot(epocas, val_acc, 'b', label='exatidao - validação')
plt.title('Valor da métrica – treinamento e validação')
plt.xlabel('Épocas')
plt.ylabel('Exatidao')
plt.legend()
plt.show()

### Exercício #11: Avaliação da RNA

Calcule o valor da função de custo e da métrica para os dados de teste.

In [ ]:
# Para você fazer: Avaliar a RNA para os dados de teste



**Saída esperada:**

    55/55 [==============================] - 0s 2ms/step - loss: 2.3789 - accuracy: 0.3333

### Exercício #12: Teste da RNA para alguns exemplos de teste

Na célula abaixo gere um lote de dados de teste, calcule a previsão da RNA para esses dados, determine as classes a partir das previsões da RNA e faça um gráfico dos resultados.

In [ ]:
# Para você fazer:

# Gera um lote de exemplos de dados do conjunto de teste
for x, y in test_ds.take(1):
    # calcula previsão da RNA
    y_prev = rna.predict(x)
    
    # Identifica classes dos exemplos
    y_prev = np.argmax(rna.predict(x), axis=1)
    
    # Mostra saídas real e prevista
    plt.figure(figsize=(16,5))
    plt.plot(y, 'ro', label='Classe real')
    plt.plot(y_prev, 'bo', label='Classe prevista')
    plt.legend()
    plt.show()

- Observe que cada círculo vermelho que aparece na figura representa um erro de classificação.

## 7. Para você fazer

Como você deve ter percebido os resultados apresentam muito overfitting e são muito insatisfatórios.

Pense no que você pode fazer para melhorar os dados. Algumas dicas:

1. Usar transformações diferentes nos dados de entrada, tais como, usar intervalos de segmentação diferentes, normalizar valores numéricos de outra forma, agrupar classes de dados categóricos, usar outras características etc.

2. Tentar usar regularização L1, L2 e/ou dropout.

3. Alterar a configuração da RNA, tal como, usar menos/mais camadas, usar menos/mais neurônios, usar outra configuração de rede (residual por exemplo) etc.

**Implemente algumas dessas modificações sugeridas no seu trabalho. A sua nota nesse trabalho depende do desempenho que você conseguir obter com a sua RNA nos dados de teste.**